<a href="https://colab.research.google.com/github/simonbustamante/mit-algoritmos-de-regresion/blob/master/Caso_de_Estudio_2_2_Brecha_salarial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.2 - Brecha salarial entre hombres y mujeres

Configuración del *notebook*:

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [1]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1U5UQdm43w9zh2w1VDxevQsb-vp3T7YCj'})
data_drop.GetContentFile('pay.discrimination.Rdata')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


In [3]:
! pip install rpy2==3.5.1
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
! pip install rpy2==3.5.1

## Datos


In [5]:
%%R
# Cargar la base de datos
load(file="pay.discrimination.Rdata")

# Mostrar las variables de la base de datos
class(data)
str(data)

# Adjuntar la base de datos al espacio de trabajo
attach(data)

# Mostrar dimensiones de la base de datos
dims  <- dim(data)
cat('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)

'data.frame':	3835 obs. of  12 variables:
 $ female: num  0 0 0 0 1 0 1 0 0 0 ...
 $ cg    : num  0 1 0 1 1 0 0 0 0 0 ...
 $ sc    : num  0 0 1 0 0 1 1 0 0 1 ...
 $ hsg   : num  1 0 0 0 0 0 0 1 1 0 ...
 $ mw    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne    : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1  : num  33 27 13 2 15 6.5 6 25 14 26 ...
 $ exp2  : num  10.89 7.29 1.69 0.04 2.25 ...
 $ exp3  : num  35.937 19.683 2.197 0.008 3.375 ...
 $ wage  : num  11.66 12.82 5.78 12.47 18.52 ...

Dimensiones de la base de datos: 3835, 12 



Elaboración de una tabla con la media de cada variable, controlando por género:

In [6]:
%%R

stats.female  <- as.matrix(apply(data[female==1,], 2, mean))
stats.male    <- as.matrix(apply(data[female==0,], 2, mean))
stats         <- cbind(stats.male, stats.female)

colnames(stats) = c("media hombres", "media mujeres")
print(stats,digits=2)

       media hombres media mujeres
female          0.00          1.00
cg              0.35          0.41
sc              0.30          0.35
hsg             0.34          0.24
mw              0.28          0.29
so              0.24          0.26
we              0.22          0.20
ne              0.26          0.26
exp1           13.58         13.04
exp2            2.59          2.45
exp3            5.96          5.60
wage           16.12         14.72


## Metodología

### Regresión lineal

In [7]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Modelo básico:

In [8]:
%%R
# Regresión lineal del salario
fmla1 <- wage ~ female + sc + cg + mw + so + we + exp1 + exp2 + exp3
# Ejecución de OLS, obtener coeficientes, errores estándar e intervalos de confianza del 95%
full.fit1 <- lm(fmla1, data=data)
est1 <- summary(full.fit1) $coef[2,1:2]
ci1 <-  confint(full.fit1)[2,]

Modelo flexible:

In [9]:
%%R
# Regresión lineal: especificación cuadrática
fmla2     <-  wage ~  female + (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)^2

# Ejecución de OLS, obtener coeficientes, errores estándar e intervalos de confianza del 95%
full.fit2 <- lm(fmla2, data=data)
est2      <- summary(full.fit2)$coef[2,1:2]
ci2       <- confint(full.fit2)[2,]

In [10]:
%%R
# Resumen de las especificaciones lineal y cuadrática
table1     <- matrix(0, 2, 4)
table1[1,] <- c(est1,ci1)
table1[2,] <- c(est2,ci2)

# Asignando nombres a filas y columnas
colnames(table1) <- c("Estimación (beta)", "Error estándar", "Inter. Conf. Inf.", "Inter. Conf. Sup.")
rownames(table1) <- c("reg basica", "reg flexible")

### Extracción parcial

Especificación lineal (modelo básico)

In [11]:
%%R
# Regresión lineal de y (resultado) en las covariables
fmla1.y <- wage ~  sc+ cg+ mw + so + we + exp1 + exp2 + exp3

# Regresión lineal de d (regresor objetivo/tratamiento) en las covariables
fmla1.d <- female ~  sc+ cg+ mw + so + we + exp1 + exp2 + exp3

# Residuales de la regresión de y
t.Y    <- lm(fmla1.y, data=data)$res

# Residuales de la regresión de d
t.D    <-  lm(fmla1.d, data=data)$res


# Ejecución de OLS, obtener coeficientes, errores estándar e intervalos de confianza del 95%
partial.fit1   <- lm(t.Y~t.D)
partial.est1   <- summary(partial.fit1)$coef[2,1:2]
partial.ci1    <- confint(partial.fit1)[2,]

Especificación cuadrática (modelo flexible)

In [12]:
%%R
fmla2.y  <- wage ~  (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)^2
fmla2.d  <- female ~ (sc+ cg+ mw + so + we + exp1 + exp2 + exp3)^2

# Obtener residuales de la regresión lineal
t.Y  <- lm(fmla2.y, data=data)$res
t.D  <- lm(fmla2.d, data=data)$res

# Regresión de los residuales entre ellos para obtener el resultado de la extracción parcial
partial.fit2  <-  lm(t.Y~t.D)
partial.est2  <-  summary(partial.fit2)$coef[2,1:2]
partial.ci2   <-  confint(partial.fit2)[2,]


# Crear tabla para recoger los resultados
table2     <- matrix(0, 4, 2)
table2[1,] <- c(est1)
table2[2,] <- c(est2)
table2[3,] <- c(partial.est1)
table2[4,] <- c(partial.est2)

# Asignar nombres de filas y columnas
colnames(table2) <- c("Estimación (beta)", "Error estándar")
rownames(table2) <- c("reg basica", "reg flexible", "reg basica con extr. parcial", "reg flexible con extr. parcial")

## Resultados

In [13]:
%%R
# Mostrar resultados
cat('- Regresión lineal:\n',fill = TRUE)
print(table1,digits=3)

cat('\n\n- Comparación con y sin extracción parcial:\n',fill = TRUE)
print(table2,digits=3)

- Regresión lineal:

             Estimación (beta) Error estándar Inter. Conf. Inf.
reg basica               -1.83          0.425             -2.66
reg flexible             -1.88          0.425             -2.71
             Inter. Conf. Sup.
reg basica              -0.994
reg flexible            -1.047


- Comparación con y sin extracción parcial:

                               Estimación (beta) Error estándar
reg basica                                 -1.83          0.425
reg flexible                               -1.88          0.425
reg basica con extr. parcial               -1.83          0.424
reg flexible con extr. parcial             -1.88          0.423


In [14]:
%%R
print(partial.fit1)


Call:
lm(formula = t.Y ~ t.D)

Coefficients:
(Intercept)          t.D  
 -1.513e-16   -1.826e+00  

